In [1]:
import gym
import random
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import time
from gym.envs.registration import register
from IPython.display import clear_output
from collections import deque

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py
# https://github.com/openai/gym/blob/master/gym/envs/__init__.py
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

# env_name = "CartPole-v1"
# env_name = "MountainCar-v0"
# env_name = "MountainCarContinuous-v0"
# env_name = "Acrobot-v1"
# env_name = "Pendulum-v0"
env_name = "FrozenLake-v0"
env_name = "FrozenLakeNoSlip-v0"
env = gym.make(env_name)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
type(env.action_space)

Observation space: Discrete(16)
Action space: Discrete(4)


gym.spaces.discrete.Discrete

In [3]:
class Agent():
    def __init__(self, env):
        self.is_discrete = \
            type(env.action_space) == gym.spaces.discrete.Discrete
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("Action size:", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("Action range:", self.action_low, self.action_high)
        
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,
                                       self.action_high,
                                       self.action_shape)
        return action

In [4]:
class QNAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.001):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.exp_buff = deque(maxlen=1000)
        self.batch_size=50
        self.build_model()
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def build_model(self):
        tf.reset_default_graph()
        self.state_in = tf.placeholder(tf.int32, shape=[None])
        self.action_in = tf.placeholder(tf.int32, shape=[None])
        self.target_in = tf.placeholder(tf.float32, shape=[None])
        
        self.state = tf.one_hot(self.state_in, depth=self.state_size)
        self.action = tf.one_hot(self.action_in, depth=self.action_size)
        
        self.q_state = tf.layers.dense(self.state, units=self.action_size, name="q_table")
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state, self.action), axis=1)
        
        self.loss = tf.reduce_sum(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
    def get_action(self, state):
        q_state = self.sess.run(self.q_state, feed_dict={self.state_in: [state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        self.exp_buff.append(experience)
        batch = random.choices(self.exp_buff,k=self.batch_size)
        
        
        state, action, next_state, reward, done = (list(col) for col in zip(*batch))
        q_next = self.sess.run(self.q_state, feed_dict={self.state_in: next_state})
        q_next[done] = np.zeros([self.action_size])
        q_target = reward + self.discount_rate * np.max(q_next,axis=1)
        
        feed = {self.state_in: state, self.action_in: action, self.target_in: q_target}
        self.sess.run(self.optimizer, feed_dict=feed)
        
        if experience[4]:
            self.eps = self.eps * 0.99
            
    def __del__(self):
        self.sess.close()
        

In [5]:
agent = QNAgent(env)

Action size: 4
State size: 16
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [10]:
total_reward = 0
wins=0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        if state==next_state or (done and next_state!=15):
            reward=-1
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        if reward>0:
            wins+=1
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, ,wins: {}, eps: {}".format(ep,total_reward,wins,agent.eps))
        env.render()
        with tf.variable_scope("q_table", reuse=True):
            weights = agent.sess.run(tf.get_variable("kernel"))
            print(weights)
        clear_output(wait=True)

s: 15 a: 2
Episode: 99, Total reward: 96.0, ,wins: 99, eps: 0.006570483042414605
  (Right)
SFFF
FHFH
FFFH
HFFG
[[-0.24352722  0.6726038   0.31036764 -0.2409442 ]
 [ 1.0442712  -0.5426386   0.19432771 -0.348302  ]
 [ 0.3812741   0.54951686 -0.17421179 -0.17267114]
 [ 0.42762864 -0.4597154  -0.9483002  -0.34603173]
 [-0.08291665  0.69807154 -1.0859655   0.99340135]
 [-0.4089734   0.12385923 -0.38812226 -0.08604002]
 [-0.639267    0.6714836  -0.5447458   0.76443803]
 [-0.08174056 -0.46308193  0.16479224  0.21743679]
 [ 0.07923438 -1.1314865   0.6186681   0.991391  ]
 [ 1.1074028   0.7544216   0.59283805 -0.6477473 ]
 [ 1.1223603   0.190741   -0.44745332  0.6638019 ]
 [ 0.21576148 -0.22964603  0.40059114 -0.00207466]
 [ 0.14062232  0.33851022 -0.04250377  0.4162916 ]
 [-0.78666466 -0.25132748  0.6760495   0.90910596]
 [ 1.1578683  -0.27984574  0.70604247  0.7815263 ]
 [ 0.2680387  -0.13042465  0.35547388 -0.4292443 ]]
